# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless') # 창 숨기기


In [ ]:
# from user_agent import generate_user_agent, generate_navigator

# print(generate_user_agent(device_type='desktop'))
# print(generate_user_agent(os='win', device_type='desktop'))

# navigator = generate_navigator()

# random_user = generate_user_agent(os='win', device_type='desktop')
# print(random_user)

In [2]:
from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = options)

#driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


In [3]:
url_df = pd.read_csv("./나무문화/2020/나무문화_링크_2020_9월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(11904, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/jinidag/222104033228,추석연휴 가볼만한곳 한국민속촌 야외 당일치기 나들이,2020.09.30.
1,https://blog.naver.com/taya2765/222104032204,"동래 가볼만한 곳, 복천박물관",2020.09.30.
2,https://blog.naver.com/yeonjae1213/222104028784,배꼽 빠질 뻔했다,2020.09.30.
3,https://blog.naver.com/sunnybridge_/222104026647,"내가 사랑했던 마을,탬워스(Tamworth)",2020.09.30.
4,https://blog.naver.com/lby56/222104022581,언택트 한가위… ‘꼰대’ 삼촌이 10대 조카에게 보내는 편지,2020.09.30.
...,...,...,...
11899,https://blog.naver.com/chookk7/222076688184,“먹튀자본 MBK 김병주의 땅투기로 안산이 망가진다”,2020.09.01.
11900,https://blog.naver.com/bugman1303/222076686202,머리가 좋은 동물들,2020.09.01.
11901,https://blog.naver.com/sherry8279/222076052955,"<도시는 무엇으로 사는 가, 유현준> 도시의 다채로운 생각과...",2020.09.01.
11902,https://blog.naver.com/butterfly0724/222071335341,[경기도 미술관 카페] 호이안,2020.09.01.


In [4]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


11904


In [ ]:
# df = pd.DataFrame(columns = {'url','본문'}) # 처음 만들때를 위한 코드라 주석처리 하였음.
# df.to_csv("./나무문화/2020/나무문화_본문_2020_9월.csv", index=False, encoding='UTF-8')
# df.to_excel("./나무문화/2020/나무문화_본문_2020_9월.xlsx", index=False, encoding='UTF-8')

In [5]:
df = pd.read_csv("./나무문화/2020/나무문화_본문_2020_9월.csv", encoding='UTF-8')
df

,본문,url
0,"추석연휴 가볼만한곳 야외 나들이\n""이글은 한국민속촌으로부터 금전적 지원을 받아 작...",https://blog.naver.com/jinidag/222104033228
1,복천박물관\n부산 동래구 복천로 63\n매일 09:00-18:00 (매주 마지막주 ...,https://blog.naver.com/taya2765/222104032204
2,오늘 아침에 일어나서 엄마랑 극적 화해하고 기분 좋았다. 그런데 기상은 늦었다 10...,https://blog.naver.com/yeonjae1213/222104028784
3,호주 워킹홀리데이 상반기 시즌은 사진이 많이 없어서\n포스팅 할게 많이 없다 ㅠㅠ\...,https://blog.naver.com/sunnybridge_/222104026647
4,블로그\n공지 목록\n공지글\n글 제목 작성일\n(12)\n공지 이병렬 저서 출간 ...,https://blog.naver.com/lby56/222104022581
...,...,...
4567,"과거 커피라고는 자판기 커피가 전부인 줄 알았는데, 언젠가부터 프랜차이즈 커피 전문...",https://blog.naver.com/jongscokr/222092173601
4568,안녕하세요! 해커인사팀 채용 담당자입니다. :)\n채용 과정에 돌입하면 취업 준비생...,https://blog.naver.com/hacademia/222093107681
4569,안동여행 _ 유네스코 세계문화유산 병산서원\n#안동여행 을 계획한다면 꼭 들려봐야할...,https://blog.naver.com/outro10/222089932265
4570,#반타얀싼타페\n반타얀 BANTAYAN아일랜드 TRAVEL 가이드!\nTANON 해...,https://blog.naver.com/qtuj4iijy/222092730591


In [6]:
len(df)

4572

In [ ]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(10)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
        # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    time.sleep(1)
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("./나무문화/2020/나무문화_본문_2020_9월.csv", index=False, encoding='UTF-8')
    df.to_excel("./나무문화/2020/나무문화_본문_2020_9월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    
    display.clear_output(wait=True) # 출력지우기
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


In [ ]:
df

In [ ]:
print(len(contents))
#print(len(old_blogs))
print(len(hide_link))
